In [1]:
import pandas as pd

In [2]:
df_all = pd.read_csv("data/rmr_answers-2025-03.csv")

In [3]:
def get_state(lat, lng):
    # Bounding box for Texas
    if 25 <= lat <= 36.5 and -106.65 <= lng <= -93.5:
        return 'Texas'
    # Bounding box for California
    elif 32 <= lat <= 42 and -124.4 <= lng <= -114.1:
        return 'California'
    else:
        return 'Unknown'

df_all['state'] = df_all.apply(lambda row: get_state(row['start_stop_lat'], row['start_stop_lng']), axis=1)
df_texas = df_all[df_all['state'] == 'Texas'].copy()
df_california = df_all[df_all['state'] == 'California'].copy()

In [4]:
df_texas.shape

(100933, 31)

In [5]:
df_california.shape

(829516, 31)

In [8]:
# df_fare = df_texas[df_texas["question_id"].str.startswith("fare_payment")].copy()
df = df_texas[(df_texas["question_id"] == "on_time_schedule_v2") | 
                    (df_texas["question_id"] == "on_time_real_time_v2") | 
                    (df_texas["question_id"] == "on_time")]

df = df.sort_values(["user_trip_id", "datetime_local_timezone"])

# Create a chain identifier.
# if 'followed_up_from_question_title' is NaN, start new chain
df["chain_id"] = df.groupby("user_trip_id")["followed_up_from_question_title"]\
                             .transform(lambda x: x.isnull().cumsum())

# Chain func
def build_chain(group):
    sorted_group = group.sort_values("datetime_local_timezone")
    chain = []
    for _, row in sorted_group.iterrows():
        chain.append(row['question_title'])
        chain.append(row['answer_title'])
    return chain

chains_df = df.groupby(["user_trip_id", "chain_id"]).apply(build_chain).reset_index(name='chain')

# print("Chains of Questions and Answers:")
# print(chains_df)

chains_df.to_csv("texas_chains.csv")

Chains of Questions and Answers:
                            user_trip_id  chain_id  \
0   06a166b6-70f2-41de-8e2d-d3958997f6aa         1   
1   0a59cf27-be0e-4a47-abf3-9a98cce6858d         1   
2   0b4b1704-057a-4b5e-be27-9e2fad848c84         1   
3   0b86affb-87d7-415d-8a02-227220f592cc         1   
4   0db4bf52-fba6-42f8-9818-0e64d98bc395         1   
5   0eaf6f6b-442b-4971-9254-23f67bc03c12         1   
6   16ff3382-13e0-456c-8b59-5f1e0801150d         1   
7   193a2f74-dba4-4aa4-ba68-645764eb7b15         1   
8   1a4f9b2d-0d36-4eda-a198-2a7e1f063c27         1   
9   320bfae6-0b78-4064-819d-607f2b2651d4         1   
10  3c873c86-6933-4998-ae95-06c1a890421a         1   
11  3c873c86-6933-4998-ae95-06c1a890421a         2   
12  51920bc6-72d5-4029-8573-5b4707706bde         1   
13  523e4044-ef26-4924-b4d5-03ccc1dd56d7         1   
14  5cbd5dca-3a55-4a7a-b661-a7ca6d190401         1   
15  6e1607d0-0388-4d9b-97f4-0e7e242a658d         1   
16  7144407f-807e-4d6b-90c8-74ce8ab64150         

C:\Users\J-Lee\AppData\Local\Temp\ipykernel_6100\1489037058.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  chains_df = df_fare.groupby(["user_trip_id", "chain_id"]).apply(build_chain).reset_index(name='chain')


In [9]:
chains_df["chain_tuple"] = chains_df["chain"].apply(tuple)

unique_chain_counts = chains_df["chain_tuple"].value_counts().reset_index()
unique_chain_counts.columns = ['chain_combo', 'count']

print("Unique chain combinations and their counts:")
print(unique_chain_counts)
unique_chain_counts.to_csv("result/ontime/202503/texas_unique chain.csv")

Unique chain combinations and their counts:
                                         chain_combo  count
0  (Did your [vehicle_name] come when you thought...     11
1  (Did your [vehicle_name] come when you thought...     10
2  (Did your [vehicle_name] come when the real-ti...      7
3  (Did your [vehicle_name] come when the real-ti...      2
4  (Did your [vehicle_name] come when the real-ti...      2
5  (Did your [vehicle_name] come when you thought...      2
